### Day 1

In [21]:
data = [*map(int, open('data/day1.txt').readlines())]
r = [x + y + z for x, y, z in zip(data, data[1:], data[2:])]
print('Part 1:', sum(y > x for x, y in zip(data, data[1:])))
print('Part 2:', sum(y > x for x, y in zip(r, r[1:])))

### Day 2

In [22]:
h = v = 0
for k in open('data/day2.txt').read().splitlines():
    a, b = k.split()
    b = int(b)
    if a == "forward":
        h += b
    elif a == "down":
        v -= b
    else:
        v += b

print('Part 1:', abs(h) * abs(v))

h = v = a = 0
for k in open('data/day2.txt').read().splitlines():
    x, b = k.split()
    b = int(b)
    if x == "forward":
        h += b
        v += a * b
    elif x == "down":
        a -= b
    else:
        a += b

print('Part 2:', abs(h) * abs(v))

### Day 3

In [23]:
b = list(map(list, open('data/day3.txt').readlines()))
z = list(map(list, zip(*b)))
i = ("".join(max(k, key = k.count) for k in z))
j = ("".join(min(k, key = k.count) for k in z))

print('Part 1:', int(i, 2) * int(j, 2))

def cut(items, criteria, index = 0):
    v = [item[index] for item in items]
    q = criteria(v, key = lambda x: (v.count(x), x))
    items = [item for item in items if item[index] == q]
    if len(items) == 1:
        return items[0]
    else:
        return cut(items, criteria, index + 1)

ox = int("".join(cut(b, max)), 2)
co = int("".join(cut(b, min)), 2)
print('Part 2:',ox * co)

### Day 4

In [1]:
import pandas as pd
import numpy as np

with open('data/day4.txt') as data:
    input_data = [line.strip() for line in data]

nums = input_data[0]
nums = list(map(int, nums.split(',')))

boards = np.array_split(input_data[1:],(len(input_data)-1)/6)
boards = [np.array(i[1:]) for i in boards]
boards_dict = {}
for i, j in enumerate(boards):
    boards_dict["boards_"+str(i+1)] = [np.array(list(map(int,k.split()))+[0]) for k in j]
    boards_dict["boards_"+str(i+1)] = np.vstack((boards_dict["boards_"+str(i+1)], np.array([[0,0,0,0,0,0]])))
boards_dict_copy = boards_dict.copy()

In [2]:
def get_index(mat, num):
    return np.where(mat[:5,:5] == num)
# res = get_index(np.array(boards_dict_copy['boards_1']), nums[0])

def update_mat(mat,result):
    if result[0].size != 0:
        mat[result[0][0],5] += 1
        mat[5,result[1][0]] += 1
    return mat
# update_mat(boards_dict_copy['boards_1'],res)

def check_bingo(mat):
    if mat[:,5].max() == 5:
        return True
    elif mat[5,:].max() == 5:
        return True
    else:
        return False
# check_bingo(boards_dict_copy['boards_1'])


In [5]:
def compute_winner(boards_dict, nums):
    slice_num_list = []
    for i in nums:
        slice_num_list.append(i)
        for key, value in boards_dict.items():
            idx = get_index(value,i)
            temp = update_mat(value, idx)
            boards_dict[key] = temp
            if check_bingo(temp):
                return i, temp, key, idx, slice_num_list
    return None
# res = compute_winner(boards_dict_copy, nums)

def cal_score(res):
    return res[0] * (res[1][:5,:5].sum() - sum(list(set(res[1][:5,:5].flatten()).intersection(set(res[4])))))
# cal_score(res)

--------------------------------------

In [5]:
def compute_last(boards_dict, nums):
    key_copy = tuple(boards_dict.keys())
    slice_num_list = []
    key_list = []
    for i in nums:
        slice_num_list.append(i)
        for key in key_copy:
            idx = get_index(boards_dict[key],i)
            temp = update_mat(boards_dict[key], idx)
            boards_dict[key] = temp
            if check_bingo(temp):
                key_list.append(key) if key not in key_list else key_list
                if len(key_list) == 100:
                    return i, boards_dict, slice_num_list, key_list
# res = compute_last(boards_dict_copy, nums)

def cal_score(res, boards_dict):
    return res[0] * (boards_dict[res[3][-1]][:5,:5].sum() - \
          sum(list(set(res[1][res[3][-1]][:5,:5].flatten()).intersection(set(res[2])))))
# cal_score(res, boards_dict)

### Day 5

In [24]:
def cal_points(p2):
    with open('data/day5.txt') as f: 
        data = [list(map(int, line.strip().replace('->', ',').split(',')))
                for line in f]

    nx = max(max([(c[0], c[2]) for c in data])) + 1
    ny = max(max([(c[1], c[3]) for c in data])) + 1

    danger_zone = [[0 for i in range(nx)] for j in range(ny)]

    for x1, y1, x2, y2 in data:
        xmin, dx = min(x1, x2), abs(x2 - x1)
        ymin, dy = min(y1, y2), abs(y2 - y1)
        
        if x1 == x2:
            for i in range(dy + 1):
                danger_zone[i + ymin][x1] += 1
        elif y1 == y2:
            for i in range(dx + 1):
                danger_zone[y1][i + xmin] += 1
        elif p2:
            cx = 1 if x2 > x1 else -1
            cy = 1 if y2 > y1 else -1
            for i in range(dx + 1):
                danger_zone[cy*i + y1][cx*i + x1] += 1

    return sum([1 for line in danger_zone for num in line if num > 1])

print('Part 1:', cal_points(0))
print('Part 2:', cal_points(1))

### Day 6

In [18]:
%%time
from collections import defaultdict
with open('data/day6.txt') as f: 
    data = [list(map(int, line.strip().split(','))) for line in f][0]

def count_of_fish(data, days): 
    data_dict = defaultdict(int)
    for i in data:
        data_dict[i] += 1
    
    for i in range(days):
        zeros = data_dict[0]
        data_dict[0] = 0
        for idx in range(1,len(data_dict)):
            data_dict[idx-1] += data_dict[idx]
            data_dict[idx] = 0
        data_dict[6] += zeros
        data_dict[8] += zeros
    return sum(data_dict.values())

print('Part 1:', count_of_fish(data,80))
print('Part 2:', count_of_fish(data,256))

### Day 7

In [25]:
k = [list(map(int, i.split(","))) for i in open('data/day7.txt').readlines()]
p1, p2 = float("inf"), float("inf")
g = lambda x: x * (x + 1) // 2

for i in range(min(k[0]), max(k[0]) + 1):
    t = sum(abs(x - i) for x in k[0])
    p1 = min(p1, t)

for i in range(min(k[0]), max(k[0]) + 1):
    t = sum(g(abs(x - i)) for x in k[0])
    p2 = min(p2, t)
    
print('Part 1:', p1)
print('Part 2:', p2)

### Day 8

Rules to identify numbers:
- 1,4,7,8 can be identified easily based on **length** of characters.
- 2,3,5 each have **five** characters each and they can be identified as follows:
    - All the characters 1 has will be present in 3 and not in 2 or 5. 
    - 5 has 3 overlapping characters with 4
    - 2 is the last one in the set
- 0,6,9 each have **six** characters each and they can be identified as follows:
    - All the characters 4 has will be present in 9
    - Out of the remaining 0,6 if all the characters of 1 are present then it is 0 or else 6
    


In [36]:
%%time
with open('data/day8.txt') as f: 
    data = [line.strip().split(' | ') for line in f]
    
def part_1(data):
    len_list = [[len(j) for j in i.split(' ')] for i in [j[1] for j in data]]
    count = 0
    for i in len_list:
        for j in i:
            if j in [2,3,4,7]:
                count += 1
                
    return count

def part_2(data):
    count = 0
    for i in data:
        inp, out = sorted(i[0].split(' '), key=len), i[1].split(' ')
        inp = [''.join(sorted(char)) for char in inp]
        out = [''.join(sorted(char)) for char in out]

        map_dict = {}
        for j in inp:
            if len(j) == 2:
                map_dict[j] = 1
                temp_1 = j
            elif len(j) == 3:
                map_dict[j] = 7
            elif len(j) == 4:
                map_dict[j] = 4
                temp_2 = j
            elif len(j) == 7:
                map_dict[j] = 8
            elif len(j) == 5 and set(temp_1).issubset(set(j)):
                map_dict[j] = 3
            elif len(j) ==5 and len(set(temp_2) & set(j)) == 3:
                map_dict[j] = 5
            elif len(j) == 5:
                map_dict[j] = 2
            elif len(j) == 6 and set(temp_2).issubset(set(j)):
                map_dict[j] = 9
            elif len(j) == 6 and set(temp_1).issubset(set(j)):
                map_dict[j] = 0
            else:
                map_dict[j] = 6

        count += int("".join(map(str, [map_dict[x] for x in out])))
        
    return count

print('Part 1:', part_1(data))
print('Part 2:', part_2(data))

Part 1: 310
Part 2: 915941
CPU times: user 5.09 ms, sys: 1.73 ms, total: 6.82 ms
Wall time: 5.47 ms


### Day 9

In [40]:
%%time
import heapq
import math

data = [[int(item) for item in line.strip()] for line in open('data/day9.txt').readlines()]
rows, cols = len(data), len(data[0])

def get_neighbors(i, j):
    neighbors = []
    if (i > 0):
        neighbors.append((i - 1, j))
    if (j > 0):
        neighbors.append((i, j - 1))
    if (i + 1 < rows):
        neighbors.append((i + 1, j))
    if (j + 1 < cols):
        neighbors.append((i, j + 1))

    return neighbors

low_points = [(i, j) for i, row in enumerate(data) for j, height in enumerate(row) 
              if all(height < data[di][dj] for di, dj in get_neighbors(i, j))]

print('Part 1:', sum([data[i][j] for i,j in low_points]) + len(low_points))

def get_area(i, j):
    if (data[i][j] == 9 or data[i][j] == -1):
        return 0
    data[i][j] = -1
    return 1 + sum(get_area(di, dj) for di, dj in get_neighbors(i, j))

highest_areas = []
for i, j in low_points:
    area = get_area(i, j)
    heapq.heappush(highest_areas, area)
    if (len(highest_areas) > 3):
        heapq.heappop(highest_areas)
        
print('Part 2:', math.prod(highest_areas))

In [28]:
###Awesome solution by a top coder friend.
import sys
import math
import networkx

depths = {(x, y): int(depth) for y, ln in enumerate(open('data/day9.txt').readlines()) if ln.strip()
        for x, depth in enumerate(ln.strip())}

def neighbours(x, y):
    yield (x - 1, y)
    yield (x + 1, y)
    yield (x, y - 1)
    yield (x, y + 1)

print("Part 1:", sum(depth + 1 for (x, y), depth in depths.items() 
        if all(depth < depths.get(neighbour, math.inf) for neighbour in neighbours(x, y))))

g = networkx.Graph()
for (x, y), depth in depths.items():
    if depth == 9:
        continue
    for neighbour in neighbours(x, y):
        if depths.get(neighbour, 9) != 9:
            g.add_edge((x, y), neighbour)

basins = sorted(len(basin) for basin in networkx.connected_components(g))

print("Part 2:", basins[-3] * basins[-2] * basins[-1])

### Day 10

In [1]:
import math

my_input = [l.strip() for l in open('data/day10.txt').readlines()]

points = {')': 3, ']': 57, '}': 1197, '>': 25137, '(': 1, '[': 2, '{': 3, '<': 4}
openings = {'(': ')', '[': ']', '<': '>', '{': '}'}
invalids = []
bracket_stacks = []
for line in my_input:
    bracket_stack = []
    for c in line:
        if c in openings:
            bracket_stack.append(c)
        else:
            if c != openings[bracket_stack[-1]]:
                invalids.append(c)
                break
            bracket_stack.pop()
    else:
        bracket_stacks.append(bracket_stack)

scores = []
for bc in bracket_stacks:
    score = 0
    for c in bc[::-1]:
        score *= 5
        score += points[c]
    scores.append(score)

print("Part 1:", sum([points[x] for x in invalids]))
print("Part 2:", sorted(scores)[len(scores)//2])

### Day 11

In [19]:
import numpy as np
with open('data/day11.txt') as f:
    data = np.array([[int(x) for x in line.strip()] for line in f])

def neighbours(row, col):
    adj = [(row-1,col),(row-1,col-1),(row-1,col+1),(row, col-1), 
           (row, col+1),(row+1, col-1), (row+1,col), (row+1,col+1)]
    return [(r,c) for r,c in adj if r >=0 and c>=0 and r<=9 and c<=9]

def increment(x,y, data, flashed):
    for i,j in neighbours(x,y): 
        if not flashed[i][j]:
            data[i][j] += 1
    return data

def execute(data, steps):
    temp = data.copy()
    flashes, step = 0, 1
    while step <= steps:
        temp += 1
        has_flashed = np.zeros_like(temp).astype(bool)
        while np.argwhere(temp > 9).shape[0] > 0:
            idx = np.argwhere(temp > 9)
            for i in idx:
                has_flashed[i[0],i[1]] = True
                temp[i[0],i[1]] = 0
                increment(i[0],i[1], temp, has_flashed)
        flashes += has_flashed.sum()
        if has_flashed.sum() == 100:
            return step
            break
        step += 1
    return flashes

print("Part 1:", execute(data,100))
print("Part 2:", execute(data,250))

### Day 12

In [1]:
from collections import defaultdict
lines = [l.strip() for l in open('data/day12.txt').readlines()]
adj = defaultdict(list)

for line in lines:
    a, b = line.split("-")
    adj[a].append(b)
    adj[b].append(a)
    
def paths(cur: str, seen, double):
    if cur == 'end':
        return 1
    if cur == "start" and seen:
        return 0
    if cur.islower() and cur in seen:
        if double is None:
            double = cur
        else:
            return 0
    seen = seen | {cur}
    out = 0
    for thing in adj[cur]:
        out += paths(thing, seen, double)
    return out

print("Part 1:", paths("start", set(), 0))
print("Part 2:", paths("start", set(), None))

Part 1: 4749
Part 2: 123054


### Day 13

In [28]:
with open('data/day13.txt') as f:
    grid, folds = process(f.read().split('\n\n'))

def process(parts):
    grid = set()
    for line in parts[0].split('\n'):
        a, b = [int(n) for n in line.strip().split(',')]
        grid.add((a, b))
    folds = []
    for line in parts[1].strip().split('\n'):
        xy, n = line.strip().split(' ')[2].split('=')
        n = int(n)
        folds.append((xy, n))
    return grid, folds

def fold(grid, xy, n):
    if xy == 'x':
        return {(x, y) for x, y in grid if x < n} | {(n - (x - n), y) for x, y in grid if x > n}
    else:
        return {(x, y) for x, y in grid if y < n} | {(x, n - (y - n)) for x, y in grid if y > n}

def display_grid():
    max_x = max([pos[0] for pos in grid])
    max_y = max([pos[1] for pos in grid])
    
    for y in range(max_y + 1):
        for x in range(max_x + 1):
            if (x, y) in grid:
                print('█', end='')
            else:
                print(' ', end='')
        print()

xy, n = folds[0]
print("Part 1:", len(fold(grid, xy, n)))

for xy, n in folds:
    grid = fold(grid, xy, n)
    
print("Part 2:", display_grid())

Part 1: 842
███  ████ █  █ ███   ██    ██ ████ █  █
█  █ █    █ █  █  █ █  █    █    █ █  █
███  ███  ██   █  █ █       █   █  █  █
█  █ █    █ █  ███  █       █  █   █  █
█  █ █    █ █  █ █  █  █ █  █ █    █  █
███  █    █  █ █  █  ██   ██  ████  ██ 
Part 2: None


### Day 14

In [5]:
from collections import Counter, defaultdict
def run(steps):
    with open('data/day14.txt') as f:
        polymer, rules = f.read().split('\n\n')

    rules = {rule.split(" -> ")[0]: rule.split(" -> ")[1] for rule in rules.split("\n")}
    pairs = Counter((x+y) for x, y in zip(polymer, polymer[1:]))
    
    for _ in range(steps):
        new_pairs = Counter()
        for k, v in pairs.items():
            a, b = list(k)
            mid = rules[k]
            new_pairs[a+mid] += v
            new_pairs[mid+b] += v

        pairs = new_pairs

    total_lets = Counter()
    for k, v in pairs.items():
        total_lets[k[0]] += v
    total_lets[polymer[-1]] += 1

    return total_lets.most_common()[0][1] - total_lets.most_common()[-1][1]
    
print("Part 1:", run(10))
print("Part 2:", run(40))

Part 1: 2740
Part 2: 2959788056211


### Day 15

In [42]:
import heapq
from collections import defaultdict
from math import inf as INFINITY

with open('data/day15.txt') as f:
    data = [[int(i) for i in line.strip()] for line in f]
    
def neighbours(r, c, h, w):
    for dr, dc in ((1, 0), (-1, 0), (0, 1), (0, -1)):
        rr, cc = (r + dr, c + dc)
        if 0 <= rr < w and 0 <= cc < h:
            yield rr, cc
            
def run(grid):
    h, w = len(grid), len(grid[0])
    source = (0, 0)
    destination = (h - 1, w - 1)

    queue = [(0, source)]
    mindist = defaultdict(lambda: INFINITY, {source: 0})
    visited = set()

    while queue:
        dist, node = heapq.heappop(queue)
        if node == destination:
            return dist
        if node in visited:
            continue
        visited.add(node)
        r, c = node

        # For each unvisited neighbor of this node...
        for neighbour in filter(lambda n: n not in visited, neighbours(r, c, h, w)):
            nr, nc  = neighbour
            newdist = dist + grid[nr][nc]
            if newdist < mindist[neighbour]:
                mindist[neighbour] = newdist
                heapq.heappush(queue, (newdist, neighbour))

    return INFINITY

print('Part 1:', run(data))

tilew, tileh = len(data), len(data[0])

for _ in range(4):
    for row in data:
        tail = row[-tilew:]
        row.extend((x + 1) if x < 9 else 1 for x in tail)
        
for _ in range(4):
    for row in data[-tileh:]:
        row = [(x + 1) if x < 9 else 1 for x in row]
        data.append(row)

print('Part 2:', run(data))

Part 1: 523
Part 2: 2876


In [45]:
import networkx as nx
from itertools import product

def load_data(datafile):
    with open(datafile) as df:
        data = [list(map(int, line.strip())) for line in df.readlines()]
    return data

def gen_nx_graph(data):
    G = nx.DiGraph()
    for y,x in product(range(len(data)), range(len(data))):
        G.add_node((x, y))
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            x_, y_ = x + dx, y + dy
            if 0 <= x_ < len(data) and 0 <= y_ < len(data):
                G.add_edge((x, y), (x_, y_), weight=data[y_][x_])
    return G

def _main():

    data = load_data('data/day15.txt')
    idx = len(data) - 1

    # Part 1
    G = gen_nx_graph(data)
    print(G)
    path = nx.shortest_path(G, source=(0, 0), target=(idx, idx), weight='weight')
    risk = sum(data[y][x] for x, y in path[1:])
    print(risk)

    # Part 2
    cave = [None] * (5*len(data))
    for i, j in product(range(len(data)), range(5)):
        cave[len(data)*j + i] = data[i] * 5

    for i, j in product(range(5), range(5)):
        if i == j == 0:
            continue
        for k, l in product(range(len(data)), range(len(data))):
            x, y = i * len(data) + k, j * len(data) + l
            cave[y][x] += (i + j)
            if cave[y][x] > 9:
                cave[y][x] -= 9

    G = gen_nx_graph(cave)
    idx = len(cave) - 1
    print(G)
    path = nx.shortest_path(G, source=(0, 0), target=(idx, idx), weight='weight')
    risk = sum(cave[y][x] for x, y in path[1:])
    print(risk)

if __name__ == '__main__':
    _main()

DiGraph with 10000 nodes and 39600 edges
523
DiGraph with 250000 nodes and 998000 edges
2876


### Day 16

In [7]:
data = [x for x in open('data/day16.txt').read().strip().split('\n')][0]
parse = {'0':'0000','1':'0001','2':'0010','3':'0011','4':'0100','5':'0101','6':'0110','7':'0111','8':'1000','9':'1001',
'A':'1010','B':'1011','C':'1100','D':'1101','E':'1110','F':'1111'}

data = "".join([parse[x] for x in data])

sumofversions=0
def parse(data):
    global sumofversions
    version, tid = int(data[:3],2), int(data[3:6],2)
    sumofversions+=version

    data=data[6:]
    if tid == 4:
        t = ""
        while True:
            t += data[1:5]
            cnt = data[0]
            data=data[5:]
            if cnt == '0':
                break
        return (data, int(t,2))
    else:
        ltid= data[0]
        data=data[1:]
        spv = []
        if ltid == '0':
            l = data[:15]
            data=data[15:]
            subpacketslen = int(l, 2)
            subpackets = data[:subpacketslen]
            while subpackets:
                s,x = parse(subpackets)
                subpackets=s
                spv.append(x)
            data=data[subpacketslen:]
        else:
            l = data[:11]
            data=data[11:]
            subpacketsqty = int(l, 2)
            for i in range(subpacketsqty):
                s,x = parse(data)
                data = s
                spv.append(x)
        if tid == 0:
            return (data, sum(spv))
        if tid == 1:
            p = 1
            for x in spv:
                p*=x
            return (data,p)
        if tid == 2:
            return (data, min(spv))
        if tid == 3:
            return (data, max(spv))
        if tid == 5:
            return (data, 1 if spv[0]>spv[1] else 0)
        if tid == 6:
            return (data, 1 if spv[0]<spv[1] else 0)
        if tid == 7:
            return (data, 1 if spv[0]==spv[1] else 0)

print("Part 2:", parse(data)[1])
print("Part 1:", sumofversions)
    

Part 2: 13476220616073
Part 1: 940


### Day 17

In [28]:
from itertools import product
import re
x_min,x_max,y_min,y_max = map(int, re.findall(r'-?\d+', open('data/day17.txt').read()))

def step(pos,vel):
    x,y = pos
    vx,vy = vel
    maxy = 0
    while x <= x2+1 and y1-1 <= y:
        x, y = x + vx, y + vy
        if maxy < y: maxy = y
        if x1 <= x <= x2 and y1 <= y <= y2:
            return True,(x,y),maxy

        vx = vx-1 if vx>0 else 0
        vy -= 1
        #print(x,y,vx,vy)
    return False,(0,0),maxy

result, velValues = 0, set()
for x,y in product(range(1,x_max+1), range(y_min, -y_min)):
    hit, pos, maxy = step((0,0),(x,y))
    if hit:
        if maxy > result: result = maxy    
        velValues.add((x,y))
            
print('Part 1: ',result)
print('Part 2: ',len(velValues))

Part 1:  10296
Part 2:  2371


### Day 18

In [29]:
import itertools
import math
from functools import reduce

def add_left(x, n):
    if n is None:
        return x
    if isinstance(x, int):
        return x + n
    return [add_left(x[0], n), x[1]]

def add_right(x, n):
    if n is None:
        return x
    if isinstance(x, int):
        return x + n
    return [x[0], add_right(x[1], n)]

def explode(x, n=4):
    if isinstance(x, int):
        return False, None, x, None
    if n == 0:
        return True, x[0], 0, x[1]
    a, b = x
    exp, left, a, right = explode(a, n - 1)
    if exp:
        return True, left, [a, add_left(b, right)], None
    exp, left, b, right = explode(b, n - 1)
    if exp:
        return True, None, [add_right(a, left), b], right
    return False, None, x, None

def split(x):
    if isinstance(x, int):
        if x >= 10:
            return True, [x // 2, math.ceil(x / 2)]
        return False, x
    a, b = x
    change, a = split(a)
    if change:
        return True, [a, b]
    change, b = split(b)
    return change, [a, b]

def add(a, b):
    x = [a, b]
    while True:
        change, _, x, _ = explode(x)
        if change:
            continue
        change, x = split(x)
        if not change:
            break
    return x

def magnitude(x):
    if isinstance(x, int):
        return x
    return 3 * magnitude(x[0]) + 2 * magnitude(x[1])

with open("data/day18.txt") as f:
    lines = list(map(eval, f.read().splitlines()))
    print("Part 1:", magnitude(reduce(add, lines)))
    print("Part 2:", max(magnitude(add(a, b)) for a, b in itertools.permutations(lines, 2)))


Part 1: 3551
Part 2: 4555


### Day 19

In [30]:
from copy import deepcopy as dc
from itertools import product

scanners = []

for s in open('data/day19.txt').read().split("\n\n"):
    s = s.splitlines()[1:]
    scanners.append({tuple(map(int, k.split(","))) for k in s})

def rotations(s):
    s = dc(s)
    k = []
    for _ in range(4):
        for _ in range(4):
            k.append(s)
            s = {(z, y, -x) for x, y, z in s}
        k.append({(y, -x, z) for x, y, z in s})
        k.append({(-y, x, z) for x, y, z in s})
        s = {(x, z, -y) for x, y, z in s}
    return k

def vinv(x):
    return tuple(-k for k in x)

def vadd(x, y):
    return tuple(a + b for a, b in zip(x, y))

def vsub(x, y):
    return tuple(a - b for a, b in zip(x, y))

def calc_intersect(s1, s2):
    for s2 in rotations(s2):
        for a, b in product(s1, s2):
            off = vsub(b, a)
            c = {vsub(b, off) for b in s2}
            if len(s1 & c) >= 12:
                return (off, c)

t = set(scanners[0])
o = [(0, 0, 0)]

q = scanners[1:]

while q:
    k = calc_intersect(t, q[0])
    if k:
        off, k = k
        t |= k
        o.append(off)
        q.pop(0)
    else:
        q.append(q.pop(0))

md = 0
for a, b in product(o,o):
    md = max(md, sum(abs(x - y) for x, y in zip(a, b)))

print("Part 1:", len(t))
print("Part 2:", md)

Part 1: 483
Part 2: 14804


### Day 20

In [29]:
import numpy as np
from scipy.ndimage import convolve

alg, _, *img = open('data/day20.txt').read().splitlines()

alg = np.array([int(p=="#") for p in alg])
img = np.pad([[int(p=="#") for p in row] for row in img], (51,51))

bin2dec = 2**np.arange(9).reshape(3,3)

for i in range(50):
    img = alg[convolve(img, bin2dec)]
    if i+1 in (2, 50): print(img.sum())

5498
16014


### Day 21

In [2]:
from itertools import cycle, product
from functools import lru_cache

p1, p2 = int(open('data/day21.txt').readlines()[0][-2]), int(open('data/day21.txt').readlines()[1][-1])

def play(p1_pos, p2_pos, score_limit):
    rolls = p1_score = p2_score = 0
    die   = cycle(range(1, 101)).__next__

    while True:
        p1_pos    = (p1_pos + die() + die() + die()) % 10
        p1_score += p1_pos + 1
        rolls    += 3
        if p1_score >= score_limit:
            return rolls*p2_score

        p2_pos    = (p2_pos + die() + die() + die()) % 10
        p2_score += p2_pos + 1
        rolls    += 3

        if p2_score >= score_limit:
            return rolls*p1_score

print("Part 1: ",play(p1-1, p2-1, 1000))

rolls = tuple(map(sum, product(range(1, 4), range(1, 4), range(1, 4))))

@lru_cache(maxsize=None)
def play2(my_pos, my_score, other_pos, other_score, win_score):
    if my_score >= win_score: return 1, 0
    if other_score >= win_score: return 0, 1

    my_wins = other_wins = 0

    for roll in rolls:
        # Play one turn calculating the new score with the current roll:
        new_pos   = (my_pos + roll) % 10
        new_score = my_score + new_pos + 1

        ow, mw = play2(other_pos, other_score, new_pos, new_score, win_score) # Let the other player play, swapping the arguments:

        # Update total wins of each player:
        my_wins    += mw
        other_wins += ow

    return my_wins, other_wins

print("Part 2: ",max(play2(p1-1, 0, p2-1, 0, 21)))


Part 1:  926610
Part 2:  146854918035875


### Day 22

In [1]:
import math
import re

lines = []
for line in [line.strip() for line in open('data/day22.txt').readlines()]:
    x_min, x_max, y_min, y_max, z_min, z_max = map(int, re.findall(r'-?\d+', line.split(' ')[1]))
    lines.append((line.split(' ')[0],[x_min, x_max], [y_min, y_max], [z_min, z_max]))
    
def intersection(cube1, cube2):
    for a, b in zip(cube1, cube2):
        if a[0] > b[1] or a[1] < b[0]: return None
    return tuple((max(a[0], b[0]), min(a[1], b[1])) for a, b in zip(cube1, cube2))

def difference(cube1, cube2):
    inter = intersection(cube1, cube2)
    if not inter:
        return [cube1]
    new_cubes = []
    new_cubes.append((cube1[0], cube1[1], (cube1[2][0], inter[2][0] - 1)))
    new_cubes.append((cube1[0], cube1[1], (inter[2][1] + 1, cube1[2][1])))
    new_cubes.append(((cube1[0][0], inter[0][0] - 1), cube1[1], inter[2]))
    new_cubes.append(((inter[0][1] + 1, cube1[0][1]), cube1[1], inter[2]))
    new_cubes.append((inter[0], (cube1[1][0], inter[1][0] - 1), inter[2]))
    new_cubes.append((inter[0], (inter[1][1] + 1, cube1[1][1]), inter[2]))

    return [(x, y, z) for x, y, z in new_cubes if x[0] <= x[1] and y[0] <= y[1] and z[0] <= z[1]]

cubes = []
for idx, line in enumerate(lines):
    onoff = line[0]
    this_cube = line[1:]
    new_cubes = []
    for cube in cubes:
        new_cubes.extend(difference(cube, this_cube))
    if onoff == 'on':
        new_cubes.append(this_cube)
    cubes = new_cubes

s1 = s2 = 0
for cube in cubes:
    s1 += math.prod(max(0, min(50, cube[i][1]) - max(-50, cube[i][0]) + 1) for i in range(3))
    s2 += math.prod(cube[i][1] - cube[i][0] + 1 for i in range(3))

print("Part 1: ",s1)
print("Part 2: ",s2)

Part 1:  598616
Part 2:  1193043154475246


### Day 23

In [1]:
from functools import lru_cache

part_1 = (("D", "C"), ("B", "A"), ("C", "D"), ("A", "B"))
part_2 = (("D", "D", "D", "C"), ("B", "C", "B", "A"), ("C", "B", "A", "D"), ("A", "A", "C", "B"))

def run(lines):
    room_map = (2, 4, 6, 8)
    hall_spots = (0, 1, 3, 5, 7, 9, 10)
    destination = {"A": 0, "B": 1, "C": 2, "D": 3}
    costs = {"A": 1, "B": 10, "C": 100, "D": 1000}

    room_size = len(lines[0])
    hallway_start = tuple(None for _ in range(len(room_map) + len(hall_spots)))

    @lru_cache(maxsize=None)
    def helper(hallway, rooms):
        if rooms == (("A",) * room_size, ("B",) * room_size, ("C",) * room_size, ("D",) * room_size):
            return 0

        best_cost = float('inf')
        for i, square in enumerate(hallway):  # Move from the hallway into a room.
            if square is None:
                continue
            dest = destination[square]
            can_move = True
            for roommate in rooms[dest]:
                if roommate is not None and roommate != square:
                    # Foreigner in room: can't move there.
                    can_move = False
                    break
            if not can_move:
                continue
            offset = 1 if room_map[dest] > i else -1
            for j in range(i + offset, room_map[dest] + offset, offset):
                if hallway[j] is not None:
                    can_move = False
                    break
            if not can_move:
                continue
            none_count = sum(elem is None for elem in rooms[dest])
            new_room = (None,) * (none_count - 1) + (square,) * (room_size - none_count + 1)
            steps = none_count + abs(i - room_map[dest])
            cost = steps * costs[square]
            helper_result = helper(hallway[:i] + (None,) + hallway[i + 1:], rooms[:dest] + (new_room,)
                                   + rooms[dest + 1:])
            new_cost = cost + helper_result
            if new_cost < best_cost:
                best_cost = new_cost
        for i, room in enumerate(rooms):  # Move from a room into the hallway.
            wants_to_move = False
            for elem in room:
                if elem is not None and destination[elem] != i:
                    wants_to_move = True
            if not wants_to_move:
                continue
            none_count = sum(elem is None for elem in room)
            steps = none_count + 1
            square = room[none_count]
            for hall_destination in hall_spots:
                destination_steps = steps + abs(hall_destination - room_map[i])
                destination_cost = destination_steps * costs[square]
                blocked = False
                for j in range(min(hall_destination, room_map[i]), max(hall_destination, room_map[i])+1):
                    if hallway[j] is not None:
                        blocked = True
                        break
                if blocked:
                    continue
                new_room = (None,) * (none_count + 1) + room[none_count + 1:]
                helper_result = helper(
                    hallway[:hall_destination] + (square,) + hallway[hall_destination + 1:],
                    rooms[:i] + (new_room,) + rooms[i + 1:])
                new_cost = destination_cost + helper_result
                if new_cost < best_cost:
                    best_cost = new_cost

        return best_cost

    cost = helper(hallway_start, lines)
    return cost

print("Part 1: ",run(part_1))
print("Part 2: ",run(part_2))


Part 1:  15338
Part 2:  47064


### Day 24

In [12]:
digits = dict()
stack = list()

with open('data/day24.txt') as f:
    dig = 0
    for i, line in enumerate(f):
        _, *operands = line.rstrip().split(' ')
        if i % 18 == 4: push = operands[1] == '1'
        if i % 18 == 5: sub = int(operands[1])
        if i % 18 == 15:
            if push:
                stack.append((dig, int(operands[1])))
            else:
                sibling, add = stack.pop()
                diff = add + sub
                if diff < 0:
                    digits[sibling] = (-diff + 1, 9)
                    digits[dig] = (1, 9 + diff)
                else:
                    digits[sibling] = (1, 9 - diff)
                    digits[dig] = (1 + diff, 9)
            dig += 1

print(''.join(str(digits[d][1]) for d in sorted(digits.keys())))
print(''.join(str(digits[d][0]) for d in sorted(digits.keys())))

94992992796199
11931881141161


### Day 25

In [17]:
import numpy as np
from itertools import product

grid = np.array([list(x) for x in open('data/day25.txt').read().split('\n')])

def right(x, y):
    if y < grid.shape[1] - 1:
        return x, y + 1
    else:
        return x, 0

def down(x, y):
    if x < grid.shape[0] - 1:
        return x + 1, y
    else:
        return 0, y

def occupied(p):
    return p in herd_right or p in herd_down

herd_right = {(i, j) for i, j in product(*map(range, grid.shape)) if grid[i, j] == ">"}
herd_down = {(i, j) for i, j in product(*map(range, grid.shape)) if grid[i, j] == "v"}
herd = herd_right | herd_down

for idx in range(1000):
    herd_right = {p if occupied(r := right(*p)) else r for p in herd_right}
    herd_down = {p if occupied(d := down(*p)) else d for p in herd_down}
    new_herd = herd_right | herd_down
    if new_herd == herd:
        break
    else:
        herd = new_herd
print("Part 1: ",idx + 1)

Part 1:  530
